## Исследование надёжности заёмщиков

#### Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

#### Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Описание данных

**children — количество детей в семье**

**days_employed — общий трудовой стаж в днях**

**dob_years — возраст клиента в годах**

**education — уровень образования клиента**

**education_id — идентификатор уровня образования**

**family_status — семейное положение**

**family_status_id — идентификатор семейного положения**

**gender — пол клиента**

**income_type — тип занятости**

**debt — имел ли задолженность по возврату кредитов**

**total_income — ежемесячный доход**

**purpose — цель получения кредита**

In [49]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
from IPython.core.display import display

C:\Users\yuliy\AppData\Local\Temp\ipykernel_2400\979376918.py:4: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


**Шаг 1. Откройте файл с данными и изучите общую информацию**

In [50]:
data = pd.read_csv('./data.csv')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Посмотрим на иформацию о дата сете.

In [51]:
# 21 525 строчки и 12 столбцов
data.shape

(21525, 12)

In [52]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [53]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Сразу бросается в глаза столбец education заполнен разным шрифтом.

Столбец days_employed есть отрицательные значения, трудовой стаж 401755 слишком огромный.

Столбец family_status удалить, так как есть family_status_id, gender, income_type  нужно изменить на category.

Столбец total_income  разбить по группам доходов.

Аномальные данные children = 20, dob_years - возраст клиента 0, 


**Шаг 2. Предобработка данных**

**2.2 Обработка пропусков**

In [54]:
data.isna().sum()


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [55]:
data = data.fillna(0)
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [56]:
# посмотрим на столбец children
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [57]:
# заменим значения -1(не может быть данного значения) и 20 на 0, так как это явные выбросы
data[data['children'] == -1] = 1
data[data['children'] == 20] = 0
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

Произведена замена пропущенных значений на 0 для столбцов days_employed, total_income.

Заменили значения в столбце children -1 на 1 и 20 на 0, так как отрицательное количество детей невозможно это явные выбросы.


**2.3 Замена типа данных**

In [58]:
# сначало перевдем в нижний регистр столбец education
def lower_data():
    for column in data.columns:
        if data.dtypes[column] == 'object':
            data[column] = data[column].str.strip()
            data[column] = data[column].str.lower()

lower_data()

In [59]:
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,f,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,f,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,m,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,m,сотрудник,1,244093.050500,на покупку своего автомобиля


In [60]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21402 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21402 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21402 non-null  object 
 8   income_type       21402 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21402 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [61]:
data['days_employed'] = data['days_employed'].astype('int')
data['gender'] = data['gender'].astype('category')
data['income_type'] = data['income_type'].astype('category')
data['education'] = data['education'].astype('category')
data['family_status'] = data['family_status'].astype('category')
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   children          21525 non-null  int64   
 1   days_employed     21525 non-null  int64   
 2   dob_years         21525 non-null  int64   
 3   education         21402 non-null  category
 4   education_id      21525 non-null  int64   
 5   family_status     21402 non-null  category
 6   family_status_id  21525 non-null  int64   
 7   gender            21402 non-null  category
 8   income_type       21402 non-null  category
 9   debt              21525 non-null  int64   
 10  total_income      21525 non-null  float64 
 11  purpose           21402 non-null  object  
dtypes: category(4), float64(1), int64(6), object(1)
memory usage: 1.4+ MB


Перевела в нижний регистр столбец education, изминила тип данных days_employed на 'int', 'category' для gender, income_type, education, family_status.

**2.4 Обработка дубликатов**

In [62]:
# посчитаем сколько дубликатов
data.duplicated().sum()

np.int64(192)

In [63]:
# удалим дубликаты
data = data.drop_duplicates().reset_index(drop=True)
data.shape

(21333, 12)

Удаление cтолбцов data['education'], data['family_status'] так как строковые значения занимают больше места

In [64]:
data = data.drop(columns=['family_status', 'education'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21333 entries, 0 to 21332
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   children          21333 non-null  int64   
 1   days_employed     21333 non-null  int64   
 2   dob_years         21333 non-null  int64   
 3   education_id      21333 non-null  int64   
 4   family_status_id  21333 non-null  int64   
 5   gender            21331 non-null  category
 6   income_type       21331 non-null  category
 7   debt              21333 non-null  int64   
 8   total_income      21333 non-null  float64 
 9   purpose           21331 non-null  object  
dtypes: category(2), float64(1), int64(6), object(1)
memory usage: 1.3+ MB


Дубликаты удалены и удалены столбцы (education, family_status) т.к. дублируют данные содержащиеся в family_status_id, education_id.



**2.5 Лемматизация**

In [65]:
data.isna().sum()
data = data.dropna().reset_index(drop=True)

In [66]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21331 entries, 0 to 21330
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   children          21331 non-null  int64   
 1   days_employed     21331 non-null  int64   
 2   dob_years         21331 non-null  int64   
 3   education_id      21331 non-null  int64   
 4   family_status_id  21331 non-null  int64   
 5   gender            21331 non-null  category
 6   income_type       21331 non-null  category
 7   debt              21331 non-null  int64   
 8   total_income      21331 non-null  float64 
 9   purpose           21331 non-null  object  
dtypes: category(2), float64(1), int64(6), object(1)
memory usage: 1.3+ MB


In [68]:
m = Mystem()
for purpose in data['purpose'].unique():
    lemma = ' '.join(m.lemmatize(purpose))
    for _ in range(len(data)):
        if purpose in data.loc[_, 'purpose']:
            data.loc[_, 'purpose'] = lemma
         
print(data['purpose'].value_counts())            

purpose
недвижимость  \n                        6314
образование  \n                         3563
жилье  \n                               3408
автомобиль  \n                          3355
свадьба  \n                             2313
ремонт   жилье \n                        604
на   покупка   автомобиль \n             469
сделка   с   автомобиль \n               455
получение   высокий   образование \n     425
покупка   свой   жилье \n                425
Name: count, dtype: int64


In [69]:
# приведем все значения в столбце 'purpose' к определенным ключевым словам или лемматизированным формам
for lem_purpose in ['недвижимость', 'жилье', 'автомобиль', 'свадьба', 'образование']:
    for _ in range(len(data)):
        if lem_purpose in data.loc[_, 'purpose']:
            data.loc[_, 'purpose'] = lem_purpose

In [41]:
data['purpose'] = data['purpose'].replace(to_replace='образование  \n',
                                          value='образование').\
                                  replace(to_replace='жилье',
                                          value='недвижимость')

In [70]:
# изменим тип данных на category
data['purpose'] = data['purpose'].astype('category')

In [71]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21331 entries, 0 to 21330
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   children          21331 non-null  int64   
 1   days_employed     21331 non-null  int64   
 2   dob_years         21331 non-null  int64   
 3   education_id      21331 non-null  int64   
 4   family_status_id  21331 non-null  int64   
 5   gender            21331 non-null  category
 6   income_type       21331 non-null  category
 7   debt              21331 non-null  int64   
 8   total_income      21331 non-null  float64 
 9   purpose           21331 non-null  category
dtypes: category(3), float64(1), int64(6)
memory usage: 1.2 MB


Проведела лемматизация purpose, изменила тип данных.

Список целей займа ограничен 4-я позициями, что облегчить анализ

**2.6 Категоризация данных**

Категоризация заёмщиков по количеству детей

In [72]:
data['children'].describe()

count    21331.000000
mean         0.474005
std          0.752378
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          5.000000
Name: children, dtype: float64

In [73]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [74]:
for _ in range(len(data)):
    if data.loc[_, 'children'] == 0: data.loc[_, 'cat_children'] = 'бездетный'
    elif 1 <= data.loc[_, 'children'] <= 3: data.loc[_, 'cat_children'] = 'не более 3 детей'
    else: data.loc[_, 'cat_children'] = 'многодетный'


data['cat_children'] = data['cat_children'].astype('category')

In [75]:
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,cat_children
0,1,-8437,42,0,0,f,сотрудник,0,253875.639453,жилье,не более 3 детей
1,1,-4024,36,1,0,f,сотрудник,0,112080.014102,автомобиль,не более 3 детей
2,0,-5623,33,1,0,m,сотрудник,0,145885.952297,жилье,бездетный
3,3,-4124,32,1,0,m,сотрудник,0,267628.550329,образование,не более 3 детей
4,0,340266,53,1,1,f,пенсионер,0,158616.077870,свадьба,бездетный


Категоризация заемщиков по уровню дохода

In [76]:
data['total_income'].describe()

count    2.133100e+04
mean     1.510344e+05
std      1.098535e+05
min      0.000000e+00
25%      8.905897e+04
50%      1.357511e+05
75%      1.958319e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [77]:
for _ in range(len(data)):
    if data.loc[_, 'total_income'] <= 89062: data.loc[_, 'cat_total_income'] = 'низкий'
    elif 89062 < data.loc[_, 'total_income'] <= 135760: data.loc[_, 'cat_total_income'] = 'средний'
    elif 135760 < data.loc[_, 'total_income'] <= 195855: data.loc[_, 'cat_total_income'] = 'выше среднего'
    else: data.loc[_, 'cat_total_income'] = 'высокий'

In [78]:
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,cat_children,cat_total_income
0,1,-8437,42,0,0,f,сотрудник,0,253875.639453,жилье,не более 3 детей,высокий
1,1,-4024,36,1,0,f,сотрудник,0,112080.014102,автомобиль,не более 3 детей,средний
2,0,-5623,33,1,0,m,сотрудник,0,145885.952297,жилье,бездетный,выше среднего
3,3,-4124,32,1,0,m,сотрудник,0,267628.550329,образование,не более 3 детей,высокий
4,0,340266,53,1,1,f,пенсионер,0,158616.077870,свадьба,бездетный,выше среднего


Выполнена категоризация заемщиков по количеству детей. Критерии:

- 0 детей - бездетный

- от 1 до 3(включительно) - не более 3 детей

- больше 3 детей - многодетный

Категоризация заёмщиков по уровню дохода. Критерии:

- до 89062 - низкий

- от 89062 до 135760 - средний

- от 135760 до 195855 - выше среднего

- выше 195855 - высокий

**Шаг 3. Ответьте на вопросы**

**3.1 Есть ли зависимость между наличием детей и возвратом кредита в срок?**

In [80]:
child_pivot = data.pivot_table(values='debt', index='cat_children', aggfunc=['count', 'sum'])
child_pivot['% debt'] = child_pivot['sum'] / child_pivot['count'] * 100
child_pivot.reset_index()

C:\Users\yuliy\AppData\Local\Temp\ipykernel_2400\1746631394.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  child_pivot = data.pivot_table(values='debt', index='cat_children', aggfunc=['count', 'sum'])
C:\Users\yuliy\AppData\Local\Temp\ipykernel_2400\1746631394.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  child_pivot = data.pivot_table(values='debt', index='cat_children', aggfunc=['count', 'sum'])


,cat_children,count,sum,% debt
,,debt,debt,
0,бездетный,14091,1063,7.543822
1,многодетный,50,4,8.000000
2,не более 3 детей,7190,665,9.248957


Вероятность просроченных платежей по кредиту у бездетных заемщиков ниже чем у многодетных и заемщиков количество детей у которых не болле 3, на ~2%. Можно предположить, что зависимость между наличием детей и возвратом кредита незначительно.

**3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [83]:
family_pivot = data.pivot_table(values='debt', index='family_status_id', aggfunc=['count', 'sum'])
family_pivot['% debt'] = family_pivot['sum'] / family_pivot['count'] * 100
display(family_pivot)

,count,sum,% debt
,debt,debt,
family_status_id,,,
0,12261,927,7.560558
1,4134,385,9.313014
2,951,63,6.624606
3,1189,84,7.064760
4,2796,273,9.763948


Вероятность просроченных платежей по кредиту у заемщиков не состоящих в браке и состощих в гражданском браке(family_status_id=1) выше в сравнении с:

- овдовевшими(family_status_id=2) заемщиками на ~3%;

- заемщиками в разводе (family_status_id=3) на ~2%

- заемщиками в браке(family_status_id=0) на ~2%

**3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [84]:
income_pivot = data.pivot_table(values='debt', index='cat_total_income', aggfunc=['count', 'sum'])
income_pivot['% debt'] = income_pivot['sum'] / income_pivot['count'] * 100
display(income_pivot)

,count,sum,% debt
,debt,debt,
cat_total_income,,,
высокий,5331,381,7.146877
выше среднего,5333,478,8.963060
низкий,5334,419,7.855268
средний,5333,454,8.513032


Вероятность просроченных платежей по кредиту у заемщиков со средним доходом и выше среднего выше в сравнении с:

- заемщиками с высоким доходом на ~2%

- заемщиками с низким доходом на ~2%

**3.4 Как разные цели кредита влияют на его возврат в срок?**

In [85]:
purpose_pivot = data.pivot_table(values='debt', index='purpose', aggfunc=['count', 'sum'])
purpose_pivot['% debt'] = purpose_pivot['sum'] / purpose_pivot['count'] * 100
display(purpose_pivot)

C:\Users\yuliy\AppData\Local\Temp\ipykernel_2400\280169949.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  purpose_pivot = data.pivot_table(values='debt', index='purpose', aggfunc=['count', 'sum'])
C:\Users\yuliy\AppData\Local\Temp\ipykernel_2400\280169949.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  purpose_pivot = data.pivot_table(values='debt', index='purpose', aggfunc=['count', 'sum'])


,count,sum,% debt
,debt,debt,
purpose,,,
автомобиль,4279,400,9.347978
жилье,4437,308,6.941627
недвижимость,6314,472,7.475451
образование,3988,369,9.252758
свадьба,2313,183,7.911803


Самая низкия вероятность просроченных платежей по кредиту полученному для приобретения недвижимости.

Высокая вероятность просроченных платежей для кредитов полученных на образование и приобретение автомобиля.

**Шаг 4. Общий вывод**

В ходе исследования выявлены зависимости возникновения задолженности по кредиту от: * целей кредита * уровня дохода заемщика * наличием детей у заемщика * семейного положения заемщика